In [ ]:

! pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Question answering

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load dataset

> Add blockquote



In [ ]:
from datasets import load_dataset

dataset= load_dataset("Asmaamaghraby/EGYHistory_data", split="train[:420]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/420 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset["train"][0]

{'Question': ' لماذا كانت سياسة الرومان الاقتصادية صورة من صور الاستغلال\nالاقتصادى؟',
 'Context': 'لانهم استغلوا موارد مصر الاقتصادية لصالح روما.',
 'Answer': 'لانهم استغلوا موارد مصر الاقتصادية لصالح روما.'}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Context', 'Answer'],
        num_rows: 336
    })
    test: Dataset({
        features: ['Question', 'Context', 'Answer'],
        num_rows: 84
    })
})

## Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    contexts = []
    for c in examples["Context"]:
        if isinstance(c, str):
            contexts.append(c.strip())
        else:
            # Handle cases where context is missing or not a string
            # You can choose to skip these examples or handle them differently
            contexts.append("")  # Placeholder for missing context

    text_pairs = list(zip(questions, contexts))

    inputs = tokenizer(
        text_pairs,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answer_texts = examples["Answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer_text = answer_texts[i]
        context = contexts[i]

        # Check if context is None or empty
        if context is None or context == "":
            # Handle the case where context is missing or empty
            # You can choose to skip this example or handle it differently
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = context.find(answer_text)  # Find the start position of the answer within the context
        end_char = start_char + len(answer_text)  # Calculate the end character position
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



In [ ]:
dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 336
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 84
    })
})

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

## Train

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(dataset["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer

model_name = "aubmindlab/bert-base-arabertv2"
model = TFAutoModelForQuestionAnswering.from_pretrained("aubmindlab/bert-base-arabertv2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="historyqa_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/historyqa_model is already a clone of https://huggingface.co/Asmaamaghraby/historyqa_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=10, callbacks=[callback])

Epoch 1/10
21/21 [==============================] - 110s 3s/step - loss: 4.6890 - val_loss: 3.5754
Epoch 2/10
21/21 [==============================] - 73s 4s/step - loss: 3.3347 - val_loss: 3.0324
Epoch 3/10
21/21 [==============================] - 39s 2s/step - loss: 3.1365 - val_loss: 3.0324
Epoch 4/10
21/21 [==============================] - 45s 2s/step - loss: 3.1489 - val_loss: 3.0324
Epoch 5/10
21/21 [==============================] - 43s 2s/step - loss: 3.1397 - val_loss: 3.0324
Epoch 6/10
21/21 [==============================] - 43s 2s/step - loss: 3.1409 - val_loss: 3.0324
Epoch 7/10
21/21 [==============================] - 42s 2s/step - loss: 3.1439 - val_loss: 3.0324
Epoch 8/10
21/21 [==============================] - 43s 2s/step - loss: 3.1297 - val_loss: 3.0324
Epoch 9/10
21/21 [==============================] - 39s 2s/step - loss: 3.1456 - val_loss: 3.0324
Epoch 10/10
21/21 [==============================] - 43s 2s/step - loss: 3.1424 - val_loss: 3.0324


## Inference

In [ ]:
question = "لماذا كانت سياسة الرومان الاقتصادية صورة من صور الاستغلال?"
context = "لانهم استغلوا موارد مصر الاقتصادية لصالح روما."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="historyqa_model")
question_answerer(question=question, context=context)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at historyqa_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


{'score': 0.057623814791440964,
 'start': 6,
 'end': 46,
 'answer': 'استغلوا موارد مصر الاقتصادية لصالح روما.'}

In [50]:
question = "هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة ؟	"
Context="كان رمسيس الثاني، المعروف أيضًا باسم رمسيس الكبير، أحد أشهر فراعنة مصر القديمة. وكان محارباً وبانياً، وقاد عدة حملات عسكرية ناجحة في عهده. 2. توت عنخ آمون، المعروف أيضًا باسم الملك توت، كان فرعونًا من الأسرة الثامنة عشرة في مصر القديمة. كان عمره تسع سنوات فقط عندما أصبح فرعونًا، وانتهت فترة حكمه القصيرة عندما توفي عن عمر يناهز 18 عامًا. 3. كانت كليوباترا السابعة آخر فراعنة مصر القديمة. وكانت من الأسرة البطلمية، وعرفت بجمالها وذكائها. 4. كان تحتمس الثالث فرعونًا من الأسرة الثامنة عشرة في مصر القديمة. لقد كان قائدًا عسكريًا ناجحًا، وقام بتوسيع الإمبراطورية المصرية إلى أقصى حد. 5. نارمر هو فرعون مصر القديمة الأول، ويعتبر مؤسس الدولة المصرية. قام بتوحيد مصر العليا والسفلى، وتم تصويره كمحارب جبار على العديد من القطع الأثرية المصرية القديمة."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="historyqa_model")
question_answerer(question=question, context=context)